In [1]:
"""

Author: Fiqri Wicaksono (fiqriwicaksono@gmail.com)

"""
# import library
import pandas as pd
import numpy as np
from datetime import date, timedelta
from scipy.integrate import odeint

In [ ]:
# import cufflink library
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf
cf.go_offline(connected=True)
init_notebook_mode(connected=True)

In [2]:
# Membuat index waktu
def index_generator(start, end, diff):
    while start < end:
        yield start
        start += diff

In [ ]:
# grid of time points (days)
t = np.linspace(0, 150, 150)
# Get time index
index = list(index_generator(date(2020, 2, 14), date(2020, 2, 14) + timedelta(days=150), timedelta(days=1)))
ind = pd.DataFrame(index, columns=['Date'])

# SIR Model

In [ ]:
# persamaan diferensial SIR
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [3]:
# Jabodetabek population
N = 34e6
# based on paper https://www.nejm.org/doi/full/10.1056/NEJMoa2001316
D = 7.2 
gamma = 1 / D
# based on paper https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7074654/
r0 = 3.28 
beta = r0 * gamma
# initial conditions
S0, I0, R0 = N-12, 12, 0

In [5]:
# Initial conditions
y0 = S0, I0, R0 

In [6]:
# Integrate the SIR equations over the time grid, t.
results = odeint(SIR, y0, t, args=(N, beta, gamma))

In [7]:
# Merge dataFrame
res = pd.DataFrame(results, columns=['Susceptible', 'Infected', 'Removed'])
res = pd.merge(ind, res, left_index=True, right_index=True)
res = res.set_index('Date')

In [11]:
# plot SIR model
res.iplot(kind='scatter',
          theme='henanigans',
          xTitle='Date Since First Exposure',
          yTitle='Population',
          title='Covid-19 SIR Model')

# SEIR Model

In [12]:
# persamaan diferensial SEIR
def deriv(y, t, N, beta, gamma, sigma):
    S, E, I, R = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - sigma * E
    dIdt = sigma * E - gamma * I
    dRdt = gamma * I
    return dSdt, dEdt, dIdt, dRdt

In [14]:
# Jabodetabek population
N = 34e6
# based on paper https://www.nejm.org/doi/full/10.1056/NEJMoa2001316
incubation_duration = 5.2 
serial_interval = 7.5 
infectious_duration = serial_interval - incubation_duration
sigma = 1 / incubation_duration
gamma = 1 / infectious_duration

# based on paper https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7074654/
r0 = 3.28 
beta = r0 * gamma
# initial conditions
S0, E0, I0, R0 = N-12, 12, 0, 0

In [15]:
# initial conditions
y0 = S0, E0, I0, R0

In [16]:
# Integrate the SEIR equations over the time grid, t.
results = odeint(deriv, y0, t, args=(N, beta, gamma, sigma))

In [17]:
# Merge dataFrame
res = pd.DataFrame(results, columns=['Susceptible', 'Exposed', 'Infected', 'Removed'])
res = pd.merge(ind, res, left_index=True, right_index=True)
res = res.set_index('Date')

In [19]:
# plot SEIR Model
res.iplot(kind='scatter',
          theme='henanigans',
          xTitle='Date Since First Exposure',
          yTitle='Population',
          title='Covid-19 SEIR Model')

# SEIR Model with Logistic R0

In [26]:
# persamaan diferensial SEIR
def deriv(y, t, N, beta, gamma, sigma):
    S, E, I, R = y
    dSdt = -beta(t) * S * I / N
    dEdt = beta(t) * S * I / N - sigma * E
    dIdt = sigma * E - gamma * I
    dRdt = gamma * I
    return dSdt, dEdt, dIdt, dRdt

def logistic_R_0(t):
    return (R_0_start-R_0_end) / (1 + np.exp(-k*(-t+x0))) + R_0_end

def beta(t):
    return logistic_R_0(t) * gamma

In [21]:
# Jabodetabek population
N = 34e6
# based on paper https://www.nejm.org/doi/full/10.1056/NEJMoa2001316
incubation_duration = 5.2 
serial_interval = 7.5 
infectious_duration = serial_interval - incubation_duration
sigma = 1 / incubation_duration
gamma = 1 / infectious_duration

In [38]:
# Menentukan titik lockdown
date(2020, 2, 15) - date(2020, 4, 10)

datetime.timedelta(days=-55)

In [57]:
# parameter logistic R0
R_0_start, k, x0, R_0_end = 3.28, 0.1, 55, 0.5

In [58]:
# initial conditions
S0, E0, I0, R0 = N-12, 12, 0, 0
y0 = S0, E0, I0, R0

In [59]:
# Integrate the SEIR equations over the time grid, t.
results = odeint(deriv, y0, t, args=(N, beta, gamma, sigma))

In [60]:
# R0 over time
R0 = [logistic_R_0(i) for i in range(len(t))]
R0 = pd.DataFrame(R0)

In [61]:
# Merge results dataFrame
res = pd.DataFrame(results, columns=['Susceptible', 'Exposed', 'Infected', 'Removed'])
res = pd.merge(ind, res, left_index=True, right_index=True)
res = res.set_index('Date')

In [62]:
# plot SEIR moodel dengan logistic R0
res.iplot(kind='scatter',
          theme='henanigans',
          xTitle='Date Since First Exposure',
          yTitle='Population',
          title='Covid-19 SEIR Model with Logistic R0')

In [63]:
# plot SEIR moodel dengan logistic R0
res[['Exposed', 'Infected']].iplot(kind='scatter',
                                   theme='henanigans',
                                   xTitle='Date Since First Exposure',
                                   yTitle='Population',title='Covid-19 SEIR Model with Logistic R0')

In [65]:
# plot R0 terhadap waktu
R0.iplot(kind='scatter',
         xTitle = 'Days',
         vline='55',
         color='blue',
         theme='henanigans',
         title='R0 Over Time')